# Assignment 5 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [1]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [2]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [3]:
# word net installation:

# unmark if you want to use and need to install
# !pip install wn
# !python -m wn download omw-he:1.4

In [4]:
# word net import:

# unmark if you want to use:
# import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [5]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
#!pip install hebrew_tokenizer

In [6]:
# Hebrew tokenizer import:

# unmark if you want to use:
#import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [7]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [8]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [9]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [ ]:
# Name: Itay Vekselbum
# ID: 211871827

#### Importing libaries

Importing the following libaries / functions:
- 'LogisticRegression()'

In [10]:
from sklearn.linear_model import LogisticRegression

#### Preprocessing

The following code cell describes deleting non-Hebrew words, numbers and punctuation marks (.,:;! etc.) except whitespaces.

In [11]:
# YOUR CODE HERE
def preprocessing_cleaning(df_train):
    df_copy = df_train.copy()
    stories_lst = list(df_copy["story"])

    for index, txt in enumerate(stories_lst):
        tmp_txt = ""
        for c in txt:
            if (c >= 'א' and c <= 'ת') or c == ' ':
                tmp_txt += c
            
        stories_lst[index] = tmp_txt
    
    df_copy["story"] = stories_lst
    return df_copy

#### Vectorization

The following function describes the vectorization process on the stories. The function receives the model, an object of the model and the gender column as parameters, and performs:
1. Creating a dataframe with all tokens as columns.
2. Deleting words that less than 3 characters.
3. Converted target column from 'f'/'m' to 0/1, once 'f'=0, 'm'=1 and once 'f'=1, 'm'=0.

In [12]:
# YOUR CODE HERE
def vectorization(vectorizer, vec_data, target_col):
    X_train = pd.DataFrame(vec_data.toarray(), columns=vectorizer.get_feature_names_out())
    
    y_male = np.where(target_col == 'm', 1, 0)
    y_female = np.where(target_col == 'f', 1, 0)
    
    return X_train, y_male, y_female

#### Fit trainset

The following function fit the data after running GridSearchCV (in main code) and returns the best parameters for each model.

In [13]:
# YOUR CODE HERE
def fit_trainset(clf, X_train, y_train):
    clf.fit(X_train, y_train)
    
    best_params = clf.best_params_
    
    return best_params

#### Calculating f1_score

The following function calculates the f1_score for the model.
The function sets the parameters obtained from the grid_search function to the model, fit the data and runs the cross_val_score function which performs a series of tests and calculates the f1_score for each test (in each test, the data is divided into train and test in a different way).

In [14]:
# Add as many code cells as you need
# YOUR CODE HERE
def calc_f1_score(model, best_params, X_train, y_train):
    model.set_params(**best_params)
    model.fit(X_train, y_train)
    
    scores = cross_val_score(model, X_train, y_train, scoring="f1", cv=5)
    
    return scores, model

#### Predict (df_test)

The following function classifies the gender of the text writers based on the model that runs for df_train (including the best parameters for the model) and converts the results to 'm'/'f' (based on gender parameter).

In [15]:
def predict_df_test(vectorizer, model, model_params, df_test, X_train, y_train, gender):
    df_processed = preprocessing_cleaning(df_test)
    
    vec_test = vectorizer.transform(df_processed["story"])
    
    X_test = pd.DataFrame(vec_test.toarray(), columns=vectorizer.get_feature_names_out())
    
    model.set_params(**model_params)
    clf = model.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    if gender == 'm':
        prediction_list = np.where(y_pred == 1, 'm', 'f')
    
    else:
        prediction_list = np.where(y_pred == 0, 'm', 'f')
    
    pred_df = df_test.copy()
    pred_df["gender"] = prediction_list
    
    return pred_df

#### Printing best models

The function in the next cell calls to 'predict_df_test' function to predict the gender of the writers in df_test and prints:
- Vectorizer (with it's parameters)
- Model (with it's parameters)
- Mean of f1_score
- First 5 values of y_pred - results of the prediction process
- Last 5 values of y_pred - results of the prediction process

In [16]:
def print_best_model(vectorizer, model, y_pred, f1_score):
    print("Vectorizer: {}".format(vectorizer))
    print("Model: {}".format(model))
    print("Mean f1_score: {}".format(f1_score))
    print()
    
    gender_lst = list(y_pred)
    print("First 5 values of y_pred: {}".format(gender_lst[:5]))
    print("Last 5 values of y_pred: {}".format(gender_lst[-5:]))
    print()

#### Lists of vectorizers, models and models' parameters

The following code cell contains 3 lists:
1. vectorizers_lst - list of tuples for each vectorizer (CountVectorizer and TfIdfVectorizer) and combinations of parameters. I chose the parameters that produced the highest results during my tests.
2. models - list of models that will be used in my code. The selected models: KNeighborsClassifier, MultinomialNB, LogisticRegression, GaussianNB.
3. grid_param_lst - list of dictionaries that parameters and list of values for each model (from models list).


#### Models

- 'LogisticRegression()' - LogisticRegression is a classification algorithm in scikit-learn that is used to predict binary (two-class) outcomes. It is a linear model that uses a logistic function to model the relationship between the input features and the probability of the output class.
- 'MultinomialNB()' - Multinomial Naive Bayes algorithm is a probabilistic learning method that is mostly used in text analysis. The algorithm is based on the Bayes theorem and predicts the tag of a text. It calculates the probability of each tag for a given sample and then gives the tag with the highest probability as output.

In [17]:
vectorizers_lst = [(CountVectorizer, {'min_df' : 5, 'max_df' : 1.0, 'ngram_range' : (1, 3)}),
                   (TfidfVectorizer, {'min_df' : 5, 'max_df' : 1.0, 'ngram_range' : (1, 3)}),
                   (CountVectorizer, {'min_df' : 0.01, 'max_df' : 0.99, 'ngram_range' : (1, 2), 'max_features' : 40000}),
                   (TfidfVectorizer, {'min_df' : 0.01, 'max_df' : 0.99, 'ngram_range' : (1, 2), 'max_features' : 40000})]

models = [KNeighborsClassifier(), MultinomialNB(), LogisticRegression(), GaussianNB()]
grid_param_lst = [{'n_neighbors' : [3, 5, 7, 9, 11]},
                  {'alpha': [0.01, 0.1, 1.0, 10.0, 100.0], 'fit_prior': [True, False], 'class_prior': [None, [0.2, 0.8], [0.3, 0.7], [0.4, 0.6], [0.5, 0.5], [0.6, 0.4], [0.7, 0.3], [0.8, 0.2], [0.9, 0.1]]},
                  {'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 1.0, 10.0, 100.0], 'solver': ['lbfgs', 'newton-cg', 'liblinear'], 'class_weight': [{0: 1, 1: 1}, {0: 1, 1: 3}, {0: 1, 1: 5}], 'random_state': [41, 42]},
                  {}]

#### Running functions to test models

Performing the entire machine learning process:
- Preprocessing
- Vectorization
- Classifying with cross-validation
- Evaluating the model with f1_score
- Dividing good models and models with worse results to lists
- Printing the data of the best models

In [20]:
bad_models_lst = []
lst_y_preds = []

for vectorizer_cls, params in vectorizers_lst:
    for index, element in enumerate(models):
        df_processed = preprocessing_cleaning(df_train)

        vectorizer = vectorizer_cls(**params)
        vec_data = vectorizer.fit_transform(df_processed["story"])
        X_train, y_male, y_female = vectorization(vectorizer, vec_data, df_processed["gender"])
            
        clf = GridSearchCV(element, grid_param_lst[index], scoring="f1", cv=5, n_jobs=-1)
        best_params_male = fit_trainset(clf, X_train.copy(), y_male)
        best_params_female = fit_trainset(clf, X_train.copy(), y_female)
        
        f1_score_male, model_male = calc_f1_score(element, best_params_male, X_train, y_male)
        f1_score_female, model_female = calc_f1_score(element, best_params_female, X_train, y_female)
        f1_average = (f1_score_male.mean() + f1_score_female.mean()) / 2
            
        if f1_average >= 0.725:
            df_pred_male = predict_df_test(vectorizer, model_male, best_params_male, df_test, X_train, y_male, 'm')
            print("Male:")
            print_best_model(vectorizer, model_male, df_pred_male["gender"], f1_score_male.mean())
            
            df_pred_female = predict_df_test(vectorizer, model_female, best_params_female, df_test, X_train, y_female, 'f')
            print("Female:")
            print_best_model(vectorizer, model_female, df_pred_female["gender"], f1_score_female.mean())
            
            print("\033[1m" + "f1_average: {}".format(f1_average) + "\033[0m")

            print()
            
            lst_y_preds.append({
                'f1_average' : f1_average,
                'y_pred_male' : df_pred_male["gender"],
                'y_pred_female' : df_pred_female["gender"]
            })
            
            print("------------------------------------------------------------------------")
            
        else:
            bad_models_lst.append((element, f1_average))
        

Male:
Vectorizer: CountVectorizer(min_df=5, ngram_range=(1, 3))
Model: LogisticRegression(C=100.0, class_weight={0: 1, 1: 5}, penalty='l1',
                   random_state=41, solver='liblinear')
Mean f1_score: 0.8866496675900954

First 5 values of y_pred: ['m', 'm', 'm', 'm', 'm']
Last 5 values of y_pred: ['m', 'm', 'm', 'm', 'm']

Female:
Vectorizer: CountVectorizer(min_df=5, ngram_range=(1, 3))
Model: LogisticRegression(C=0.01, class_weight={0: 1, 1: 5}, random_state=41,
                   solver='liblinear')
Mean f1_score: 0.5745444047277307

First 5 values of y_pred: ['m', 'm', 'm', 'm', 'f']
Last 5 values of y_pred: ['m', 'm', 'm', 'm', 'm']

f1_average: 0.7305970361589131

------------------------------------------------------------------------
Male:
Vectorizer: TfidfVectorizer(min_df=5, ngram_range=(1, 3))
Model: MultinomialNB(alpha=0.01, class_prior=[0.7, 0.3])
Mean f1_score: 0.8857013607299683

First 5 values of y_pred: ['m', 'm', 'm', 'm', 'm']
Last 5 values of y_pred: ['m',

#### Choosing the best model for df_test

Here, I find the model with the best f1_score. Then, I will take the y_predicted for each target column and add it to df_predicted dataframe

In [24]:
max_f1_avg = 0

for i in lst_y_preds:
    if i['f1_average'] > max_f1_avg:
        max_f1_avg = i['f1_average']
        male_prediction_list = i['y_pred_male']
        female_prediction_list = i['y_pred_female']

#### Printing bad models

In the following code cell, I will loop over the list 'bad_models_lst' which contains models for which bad results were obtained.
In each iteration the name of the model will be printed and f1_average.

In [29]:
for i in bad_models_lst:
    print("Model: {}".format(i[0]))
    print("f1_average: {}".format(i[1]))
    print()

Model: KNeighborsClassifier(n_neighbors=3)
f1_average: 0.5279404689412412

Model: MultinomialNB(alpha=0.1, class_prior=[0.4, 0.6])
f1_average: 0.7127204386678382

Model: GaussianNB()
f1_average: 0.4770347591371338

Model: KNeighborsClassifier(n_neighbors=3)
f1_average: 0.6336353858020989

Model: GaussianNB()
f1_average: 0.4770347591371338

Model: KNeighborsClassifier(n_neighbors=3)
f1_average: 0.5405493910118219

Model: MultinomialNB(alpha=0.1, class_prior=[0.4, 0.6])
f1_average: 0.7074453578748618

Model: LogisticRegression(C=100.0, class_weight={0: 1, 1: 5}, penalty='l1',
                   random_state=41, solver='liblinear')
f1_average: 0.7203868902573942

Model: GaussianNB()
f1_average: 0.4857136786026947

Model: KNeighborsClassifier(n_neighbors=3)
f1_average: 0.6397509891794081

Model: MultinomialNB(alpha=0.1, class_prior=[0.4, 0.6])
f1_average: 0.7165114547483658

Model: GaussianNB()
f1_average: 0.4857136786026947



In [30]:
df_test_predicted = df_test.copy()

df_test_predicted["gender"] = male_prediction_list
df_test_predicted

,test_example_id,story,gender
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...,m
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""...",m
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...,m
3,3,"רגע הגיוס לצבא היה הרגע הכי משמעותי עבורי, אני...",m
4,4,אני הגעתי לברזיל ישר מקולומביה וגם אני עשיתי ע...,m
...,...,...,...
318,318,"בשנה האחרונה הרגשתי די תקוע בעבודה, השגרה הפכה...",m
319,319,אני ואילן חברים טובים מזה 20 שנה תמיד חלמנו לפ...,m
320,320,מידי יום שישי אני נוהג לנסוע בתחבורה ציבורית ס...,m
321,321,"לפני מספר חודשים, בשיא התחלואה של הגל השני, עמ...",m


### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [ ]:
df_predicted.to_csv('classification_results_male.csv',index=False)